In [ ]:
from bittensor_wallet import Wallet

from collateral_sdk import CollateralManager, Network

## Preparation


In [ ]:
manager = CollateralManager(Network.TESTNET)

#### On a miner side


In [ ]:
miner_wallet = Wallet(name='wallet0', hotkey='miner0')

In [ ]:
# Check the current stakes.
manager.subtensor_api.staking.get_stake_for_coldkey(miner_wallet.coldkeypub.ss58_address)

In [ ]:
# Use the matching miner hotkey stake as the source stake.
stake_list = manager.subtensor_api.staking.get_stake_for_coldkey(miner_wallet.coldkeypub.ss58_address)
miner_hotkey = miner_wallet.hotkey.ss58_address
miner_stake = next(
    (stake for stake in stake_list if stake.hotkey_ss58 == miner_hotkey),
    None
)
miner_stake

#### On a owner validator side


In [ ]:
vault_wallet = Wallet(name="vault", hotkey='default')

In [ ]:
# Check the current stakes.
manager.subtensor_api.staking.get_stake_for_coldkey(vault_wallet.coldkeypub.ss58_address)

In [ ]:
# Use the matching vault hotkey stake as the vault stake
stake_list = manager.subtensor_api.staking.get_stake_for_coldkey(vault_wallet.coldkeypub.ss58_address)
vault_hotkey = vault_wallet.hotkey.ss58_address
vault_stake = next(
    (stake for stake in stake_list if stake.hotkey_ss58 == vault_hotkey),
    None
)
vault_stake

## Deposit


#### On a miner side


In [ ]:
# Check the current deposit.
manager.balance_of(miner_wallet.hotkey.ss58_address)

In [ ]:
# Create an extrinsic for a stake transfer.
validator_vault_coldkey = vault_wallet.coldkeypub.ss58_address
extrinsic = manager.create_stake_transfer_extrinsic(
    amount=10 * 10**9,
    dest=validator_vault_coldkey,
    source_stake=miner_stake.hotkey_ss58,
    source_wallet=miner_wallet,
    wallet_password="REPLACE_ME"
)
extrinsic.value

In [ ]:
# Now, we can encode the extrinsic and pass to the owner.
encoded = manager.encode_extrinsic(extrinsic)

#### On a owner validator side


In [ ]:
# Now, we can decode the received extrinsic
extrinsic = manager.decode_extrinsic(encoded)
extrinsic

In [ ]:
miner_hotkey = miner_wallet.hotkey.ss58_address
manager.deposit(
    extrinsic,
    sender=miner_hotkey,
    vault_stake=vault_stake.hotkey_ss58,
    vault_wallet=vault_wallet,
    owner_address="REPLACE_ME",
    owner_private_key="REPLACE_ME"
)

In [ ]:
# Check the current deposit.
manager.balance_of(miner_hotkey)

In [ ]:
# Check the current stakes.
manager.subtensor_api.staking.get_stake_for_coldkey(miner_wallet.coldkeypub.ss58_address)

In [ ]:
# Check the current stakes of the vault wallet.
manager.subtensor_api.staking.get_stake_for_coldkey(vault_wallet.coldkeypub.ss58_address)

## Withdrawal


#### On a owner validator side


In [ ]:
manager.withdraw(
    amount=5 * 10**9,
    dest=miner_wallet.coldkeypub.ss58_address,
    source_hotkey=miner_hotkey,
    vault_stake=vault_stake.hotkey_ss58,
    vault_wallet=vault_wallet,
    owner_address="REPLACE_ME",
    owner_private_key="REPLACE_ME",
)

In [ ]:
# Check the current deposit
manager.balance_of(miner_wallet.hotkey.ss58_address)

In [ ]:
# Check the current stakes
manager.subtensor_api.staking.get_stake_for_coldkey(miner_wallet.coldkeypub.ss58_address)

In [ ]:
# Check the current stakes of the vault wallet
manager.subtensor_api.staking.get_stake_for_coldkey(vault_wallet.coldkeypub.ss58_address)